In [13]:
import numpy as np
import cv2

#### Meanshift Tracking is a object tracking algorithm which works by iteratively shifting a window (or kernel) over the video data in such a way that it converges to the regions of maximum density or intensity. initially setup ROI calculating image histograms. and itirate the algorithm on ROI and track the object

In [15]:
cap = cv2.VideoCapture('highway_cars.mp4')

# take first frame of the video
ret, first_frame = cap.read()

# setup initial location of window
# creating rectangular box for the object of interest
x =300
y = 200
h = 50
w = 80
track_window = (x, y, w, h)



# set up the ROI for tracking
roi = first_frame[x: x+h, y: y+w]
hsv_roi =  cv2.cvtColor(first_frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_criteria = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )


In [16]:
while True:
    _, frame = cap.read()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

    _, track_window = cv2.meanShift(mask, track_window, term_criteria)
    
    # draw on the image
    x, y, w, h = track_window
    cv2.rectangle(frame, (x, y), (x+w, y+h), 255, 2)

    cv2.imshow("mask [esc to quit]", mask)

    cv2.imshow("frame [esc to quit]", frame)

    key = cv2.waitKey(60)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

#### Kernelized Correlation Filter -tracking is another popular algorithm for object tracking. The ROI are used to train a correlation filter in the frequency domain. This filter represents the appearance of the target object.  In subsequent frames, the tracker uses the trained filter to detect the target object by applying correlation operations between the filter and the image patches in the vicinity of the last known location of the target.To adapt to changes in appearance due to variations in lighting, scale, or rotation, the correlation filter is continuously updated using newly observed image patches

In [18]:


# Initialize tracker with first frame and bounding box
x =300
y = 200
h = 50
w = 80
track_window = (x, y, w, h)
bbox = (x, y, w, h)  # bounding box is a tuple of (x,y,w,h)
tracker = cv2.TrackerKCF_create()

video = cv2.VideoCapture("highway_cars.mp4")

ok, frame = video.read()
ok = tracker.init(frame, bbox)

while True:
    ok, frame = video.read()
    if not ok:
        break 

    # Update tracker
    ok, bbox = tracker.update(frame)

    # Draw bounding box
    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Display result
    cv2.imshow("Tracking", frame)

    key = cv2.waitKey(60)
    if key == 27:
        break

    # # Exit if ESC key is pressed
    # if cv2.waitKey(1) & 0xFF == 27:
    #     break

video.release()
cv2.destroyAllWindows()

#### Resnet50 with Pytorch  -tracking ResNet-50 is a deep convolutional neural network architecture that has been widely used for object  detection  and tracking. initially Using  pre-trained ResNet-50 model to extract deep features and Utilize the extracted features as an appearance model for the target object. In subsequent frames, uses  techniques such as correlation filtering, template matching, or deep feature matching to locate the ROI and track it.  Periodically update the tracker using new image patches around the tracked object to account for changes in appearance 


In [22]:

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Load the pre-trained model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Initialize tracker with first frame and bounding box
tracker = cv2.TrackerKCF_create()
video = cv2.VideoCapture("highway_cars.mp4")

ok, frame = video.read()
# select ROI
bbox = cv2.selectROI(frame, False) 
ok = tracker.init(frame, bbox)

while True:
    ok, frame = video.read()
    if not ok:
        break 

    # Update tracker
    ok, bbox = tracker.update(frame)

    # Draw bounding box
    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Display result
    cv2.imshow("Tracking", frame)

    key = cv2.waitKey(60)
    if key == 27:
        break

video.release()
cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
